# Tiny GPT
Build a small GPT model using PyTorch (in mac)

## Check PyTorch instance

_Notes:_

**MPS (Metal Performance Shaders)**
* Metal Performance Shaders is an Apple framework of highly optimized GPU shaders for image processing, linear algebra, and neural networks on top of Metal.
* PyTorch, diffusers, and other ML libraries expose an mps device to offload tensor operations to the GPU via Metal/MPS on Apple silicon.


In [119]:
import torch

print(f"Torch version: {torch.__version__} and MPS availability is: {torch.backends.mps.is_available()}")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Torch device: {device.type}")

torch.set_default_device(device)
print(f"Default device: {torch.get_default_device()}")

Torch version: 2.7.0 and MPS availability is: True
Torch device: mps
Default device: mps:0


## Mathematical Explanation

A LLM model only understands numbers - based on which it calculates the probability. The model selects the next one word that has got the highest probability.
$$
\begin{aligned}
P(w_1,w_2,w_3,......,w_n) = \prod_{t=1}^{n}P(w_t|w_1,w_2,......,w_{t-1}) \\
\text{where, } w_i \text{ represents a token}
\end{aligned}
$$
E.g., suppose,
* we have the vocabulary (training set): [`very`, `tea`, `hot`, `is`, `the`]
* the first 2 words fed to the model is: [`the`, `tea`]
* the model will use the above formula of _Chain Probability_ to predict the next likely word.
    * So, it will execute `P('is'|'the','tea')`. This reads as probability of `is` given the previous words are `the` and `tea`. Similarly it will execute `P('hot'|'the','tea')` and so on. The one with the highest probability wins.
    * For a set of 4 tokens, the overall probability is calculated as
$$
\begin{aligned}
P(w_1,w_2,w_3,w_4) = P(w_1) \times P(w_2|w_1) \times P(w_3|w_1, w_2) \times P(w_4|w_1, w_2, w_3)
\end{aligned}
$$

## Training Data

In [120]:
corpus = [
    "hello friends how are you",
    "the tea is very hot",
    "my name is Sushovan",
    "the roads of Kolkata are busy",
    "it is raining in Mumbai",
    "the train is late again",
    "i love eating samosas and drinking tea",
    "holi is my favorite festival",
    "diwali brings lights and sweets",
    "india won the cricket match"
]

## Tokenize Data for the model
_Note_: Instead of using a tokenizer library (e.g. `BPE` - `Byte Pair Encoding` used in GPT or `SentencePiece` used in LLAMA), we shall be building a custom tokenizer to understand the concepts.

### Put end marker and concatenate the corpus

In [121]:
text = " ".join([s + " <END>" for s in corpus])
print(text)

hello friends how are you <END> the tea is very hot <END> my name is Sushovan <END> the roads of Kolkata are busy <END> it is raining in Mumbai <END> the train is late again <END> i love eating samosas and drinking tea <END> holi is my favorite festival <END> diwali brings lights and sweets <END> india won the cricket match <END>


### Extract the words to build the vocabulary

In [122]:
words = list(set(text.split()))
vocab_size = len(words)
print(f"Vocabulary: {words}")
print(f"Vocabulary Size: {vocab_size}")

Vocabulary: ['very', 'name', 'and', 'diwali', 'hot', 'favorite', 'brings', 'my', 'samosas', 'again', 'holi', 'raining', 'how', 'i', 'roads', 'is', 'train', 'lights', 'drinking', 'the', 'sweets', 'tea', 'are', 'in', 'love', 'it', 'of', 'hello', 'you', 'match', 'cricket', 'won', 'busy', 'Sushovan', 'india', 'late', 'festival', 'Mumbai', '<END>', 'eating', 'Kolkata', 'friends']
Vocabulary Size: 42


### Build the word index

In [123]:
word2idx = {w: idx for idx, w in enumerate(words)}
print(f"Words to Index:  {word2idx}")

idx2word = {idx: w for w, idx in word2idx.items()}
print(f"Index to Words:  {idx2word}")

Words to Index:  {'very': 0, 'name': 1, 'and': 2, 'diwali': 3, 'hot': 4, 'favorite': 5, 'brings': 6, 'my': 7, 'samosas': 8, 'again': 9, 'holi': 10, 'raining': 11, 'how': 12, 'i': 13, 'roads': 14, 'is': 15, 'train': 16, 'lights': 17, 'drinking': 18, 'the': 19, 'sweets': 20, 'tea': 21, 'are': 22, 'in': 23, 'love': 24, 'it': 25, 'of': 26, 'hello': 27, 'you': 28, 'match': 29, 'cricket': 30, 'won': 31, 'busy': 32, 'Sushovan': 33, 'india': 34, 'late': 35, 'festival': 36, 'Mumbai': 37, '<END>': 38, 'eating': 39, 'Kolkata': 40, 'friends': 41}
Index to Words:  {0: 'very', 1: 'name', 2: 'and', 3: 'diwali', 4: 'hot', 5: 'favorite', 6: 'brings', 7: 'my', 8: 'samosas', 9: 'again', 10: 'holi', 11: 'raining', 12: 'how', 13: 'i', 14: 'roads', 15: 'is', 16: 'train', 17: 'lights', 18: 'drinking', 19: 'the', 20: 'sweets', 21: 'tea', 22: 'are', 23: 'in', 24: 'love', 25: 'it', 26: 'of', 27: 'hello', 28: 'you', 29: 'match', 30: 'cricket', 31: 'won', 32: 'busy', 33: 'Sushovan', 34: 'india', 35: 'late', 36: '

### Convert to tensor
Replace each word in text with the corresponding index and fed that to a tensor

In [124]:
data = torch.tensor([word2idx[idx] for idx in text.split()], dtype=torch.long)
print(f"Tensor Data: {data}")
print(f"Data Shape: {data.shape}")

Tensor Data: tensor([27, 41, 12, 22, 28, 38, 19, 21, 15,  0,  4, 38,  7,  1, 15, 33, 38, 19,
        14, 26, 40, 22, 32, 38, 25, 15, 11, 23, 37, 38, 19, 16, 15, 35,  9, 38,
        13, 24, 39,  8,  2, 18, 21, 38, 10, 15,  7,  5, 36, 38,  3,  6, 17,  2,
        20, 38, 34, 31, 19, 30, 29, 38], device='mps:0')
Data Shape: torch.Size([62])


## Define Parameters

`block_size`: also known as context_length. It means that how many previous words the llm would refer, to predict the next word

In [125]:
block_size = 6

`embedding_dim`: Embedding Dimension

Each word in the tensor will be represented in the llm model as a dimensional vector of a defined size. Initially, random numbers are generate for each value in the dimensional vector. The llm uses this vector to predict the next word.

In [126]:
embedding_dim = 32

In [127]:
n_heads = 2  # number of Multi-head attention layer

In [128]:
n_layers = 2  # number of transformer blocks to use

In [129]:
lr = 1e-3  # learning rate

In [130]:
epochs = 1500  # number of training iterations

## Define batch function

`batch_size`: This indicates the number of sequences (or sentences) for the model to consider

In [131]:
def get_batch(batch_size=16):
    ix = torch.randint(len(data) - block_size, size=(batch_size,))  # 62 - 6 = 56 (0 to 55), 16
    # the above function gives 16 examples of random sentences
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return ix, x, y


rand_int, input_batch, output_bath = get_batch()
print(f"Data: {data}")
print(f"Batch Index: {rand_int}")
print(f"input_batch shape: {input_batch.data}")
print(f"output_bath shape: {output_bath.data}")

Data: tensor([27, 41, 12, 22, 28, 38, 19, 21, 15,  0,  4, 38,  7,  1, 15, 33, 38, 19,
        14, 26, 40, 22, 32, 38, 25, 15, 11, 23, 37, 38, 19, 16, 15, 35,  9, 38,
        13, 24, 39,  8,  2, 18, 21, 38, 10, 15,  7,  5, 36, 38,  3,  6, 17,  2,
        20, 38, 34, 31, 19, 30, 29, 38], device='mps:0')
Batch Index: tensor([11, 31, 38, 40,  9,  4,  7,  6, 31, 17, 21, 47, 10, 50, 44, 44],
       device='mps:0')
input_batch shape: tensor([[38,  7,  1, 15, 33, 38],
        [16, 15, 35,  9, 38, 13],
        [39,  8,  2, 18, 21, 38],
        [ 2, 18, 21, 38, 10, 15],
        [ 0,  4, 38,  7,  1, 15],
        [28, 38, 19, 21, 15,  0],
        [21, 15,  0,  4, 38,  7],
        [19, 21, 15,  0,  4, 38],
        [16, 15, 35,  9, 38, 13],
        [19, 14, 26, 40, 22, 32],
        [22, 32, 38, 25, 15, 11],
        [ 5, 36, 38,  3,  6, 17],
        [ 4, 38,  7,  1, 15, 33],
        [ 3,  6, 17,  2, 20, 38],
        [10, 15,  7,  5, 36, 38],
        [10, 15,  7,  5, 36, 38]], device='mps:0')
output_b

In [133]:
import numpy as np

print(f"Decoded data: \n{[idx2word[i.item()] for i in data]}")
print(f"input stack words: \n{np.array([[idx2word[i.item()] for i in b] for b in input_batch])}")
print(f"output stack words: \n{np.array([[idx2word[i.item()] for i in b] for b in output_bath])}")

Decoded data: 
['hello', 'friends', 'how', 'are', 'you', '<END>', 'the', 'tea', 'is', 'very', 'hot', '<END>', 'my', 'name', 'is', 'Sushovan', '<END>', 'the', 'roads', 'of', 'Kolkata', 'are', 'busy', '<END>', 'it', 'is', 'raining', 'in', 'Mumbai', '<END>', 'the', 'train', 'is', 'late', 'again', '<END>', 'i', 'love', 'eating', 'samosas', 'and', 'drinking', 'tea', '<END>', 'holi', 'is', 'my', 'favorite', 'festival', '<END>', 'diwali', 'brings', 'lights', 'and', 'sweets', '<END>', 'india', 'won', 'the', 'cricket', 'match', '<END>']
input stack words: 
[['<END>' 'my' 'name' 'is' 'Sushovan' '<END>']
 ['train' 'is' 'late' 'again' '<END>' 'i']
 ['eating' 'samosas' 'and' 'drinking' 'tea' '<END>']
 ['and' 'drinking' 'tea' '<END>' 'holi' 'is']
 ['very' 'hot' '<END>' 'my' 'name' 'is']
 ['you' '<END>' 'the' 'tea' 'is' 'very']
 ['tea' 'is' 'very' 'hot' '<END>' 'my']
 ['the' 'tea' 'is' 'very' 'hot' '<END>']
 ['train' 'is' 'late' 'again' '<END>' 'i']
 ['the' 'roads' 'of' 'Kolkata' 'are' 'busy']
 ['are